### TODO

* PHASE 1 Get team data from standings 
* PHASE 1 - Get and merge player data (below)
* PHASE 1 - Get player info / photo link
* PHASE 2 - Get match data / once and increment

###  LINKS

In [5]:
player_uri = 'https://www.euroleaguebasketball.net/euroleague/stats/expanded/?size=1000&viewType=advanced&seasonCode=E2022&statisticMode=perGame&seasonMode=Single&sortDirection=ascending&statistic='
real_madrid = 'https://www.euroleaguebasketball.net/euroleague/teams/real-madrid/statistics/mad/?season=2022-23&phase=All%20phases'
teams = 'https://www.euroleaguebasketball.net/euroleague/teams'
game_center = 'https://www.euroleaguebasketball.net/euroleague/game-center/'
standings = 'https://www.euroleaguebasketball.net/euroleague/standings/'
driver_path  = r'driver/chromedriver.exe'

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
from urllib.parse import urljoin

### GET PLAYER DATA

In [10]:
standings_uri = 'https://www.euroleaguebasketball.net/euroleague/standings/'
player_stats_uri = 'https://www.euroleaguebasketball.net/euroleague/stats/expanded/?size=1000&viewType=advanced&seasonCode=E2022&statisticMode=perGame&seasonMode=Single&sortDirection=ascending&statistic='
player_section_uri = 'https://www.euroleaguebasketball.net/euroleague/players/'
home = 'https://www.euroleaguebasketball.net'
driver_path = r'driver/chromedriver.exe'

In [42]:
class Extractor:
    def __init__(self, driver_path, standings_uri, player_stats_uri, player_section_uri):
        service = Service(driver_path)
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        self.driver = webdriver.Chrome(service=service, options=options)
        self.standings_uri = standings_uri
        self.player_stats_uri = player_stats_uri
        self.player_section_uri = player_section_uri        
        
    def extract_player_data(self):
        self.driver.get(self.player_stats_uri)
        time.sleep(5)
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
        rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

        header_list = []
        for cell in rows[0]:
            header_list.append(cell.text)

        data = []
        for row in rows:
            row_data = []
            for cell in row.find_all('div', {'role':'cell'}):
                if cell.find('a'):
                    span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
                    row_data.append(span.text)
                else:
                    row_data.append(cell.text)
            data.append(row_data)
        return pd.DataFrame(data[1:], columns=header_list)

    def extract_team_df(self):
        self.driver.get(self.standings_uri)
        time.sleep(5)
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
        rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

        header_list = []
        for cell in rows[0].find_all('div', attrs={'class':'complex-stat-table_cell__1lxC7'}):
            span = cell.find('span', attrs={'class':'complex-stat-table_tabletHeadName__1Jn7u'})
            if span:
                header_list.append(span.text.replace('*','').strip())
            else:
                header_list.append(cell.find('span').text.replace('*','').strip())

        data = []
        for row in rows:
            row_data = []
            for cell in row.find_all('div', {'role':'cell'}):
                if cell.find('a'):
                    span = cell.find('span', attrs={'class':'complex-stat-table_mainClubName__3IMZJ'})
                    row_data.append(span.text)
                else:
                    row_data.append(cell.text)
            data.append(row_data)
        return pd.DataFrame(data[1:], columns=header_list)
    
    def get_player_links(self):
        self.driver.get(self.player_section_uri)
        
        #Expand buttons
        load_more_class = 'all-players_loadMoreBtn__35J6M'
        load_more_button = self.driver.find_element(by="class name", value=load_more_class)
        while load_more_button: 
            load_more_button.click()
            time.sleep(2)
            try:
                load_more_button = self.driver.find_element(by="class name", value=load_more_class)
            except NoSuchElementException:
                print('all players expanded')
                load_more_button = None
                
        #Extract URI
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        player_list = soup.find('ul', attrs={'class':'all-players-list_filterList__1jH_E'})
        home = 'https://www.euroleaguebasketball.net'
        urls = []
        for li in player_list.find_all('li'):
            path = urljoin(home, li.find('a')['href'])
            urls.append(path)
        return urls

    def player_info(self, player_info_uri):
        self.driver.get(player_info_uri)
        time.sleep(3)
        
        info={}
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        spans = soup.find('h1', attrs={'class':'hero-info_fullName__IDuVE'}).find_all('span')
        name = ''
        for span in spans:
            name += f' {span.text}'
        name = name.strip()
        info['name'] = name
        info['team'] = soup.find('h2', attrs={'class':'hero-info_teamName__3g98M'}).text
        for li in soup.find('ul', attrs={'class','hero-info_dataList__2BmgP'}):
            info[li.find('span').text] = li.find('b').text
        try:
            img = soup.find('div', attrs={'class':'player-hero_imageWrap__161BQ cover-contain_ofCover__3K_IS'}).find('img')
            info['pic_link'] = img['data-srcset'].split('?')[0]
        except AttributeError:
            info['pic_link'] = None
        return info
    
    def player_info_df(self):
        uri_list = self.get_player_links()
        players = []
        for uri in uri_list[:5]:
            print(uri)
            players.append(self.player_info(uri))
        return pd.DataFrame(players)

In [43]:
extractor = Extractor(driver_path, standings_uri, player_stats_uri, player_section_uri)

In [16]:
extractor.extract_player_data()

,#,Player,Team,GP,Min,eFG%,TS%,OREB%,DREB%,REB%,AST/TO,AST-R,TO-R,2PTA-R,3PTA-R,FT-RT
0,1,ARTURS KURUCS,BKN,22,7:20,50%,50.4%,2.2%,2.2%,2.2%,1.5,30%,19.5%,15%,33%,12.5%
1,2,MATTHEW STRAZEL,ASM,20,7:20,45.2%,47.8%,0.8%,1.8%,1.2%,1.2,22.2%,18.8%,23.9%,29%,25.8%
2,3,SANDER RAIESTE,BKN,19,6:58,48%,46.4%,8%,10.8%,9.4%,0.8,13.6%,16.3%,38%,29.8%,8%
3,4,KAROLIS LUKOSIUNAS,ZAL,20,6:58,58.8%,58.8%,2.5%,4.4%,3.5%,1.5,13.6%,9.1%,6.8%,70.5%,0%
4,5,RAFI MENCO,MTA,20,5:37,44.4%,47.7%,9%,10.8%,9.9%,1.5,15.7%,10.4%,26.1%,44.4%,11.1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,204,VASILIJE MICIC,EFS,30,31:29,53.9%,59.3%,1.2%,10.8%,6.2%,1.9,24.6%,12.7%,23.8%,31.3%,31.5%
204,205,KEENAN EVANS,ZAL,17,26:13,63.4%,67.4%,1.3%,13.1%,7.2%,1.8,21.1%,11.7%,25.5%,32.2%,37.2%
205,206,WILL CLYBURN,EFS,33,33:19,52.2%,57.1%,3.9%,17.8%,11.1%,1.4,11.6%,8.5%,39.5%,31%,30.3%
206,207,MATHIAS LESSORT,PAR,33,30:22,68.9%,72.5%,14.1%,17.2%,15.7%,1,12.4%,12.9%,56.9%,0.3%,69.8%


In [17]:
extractor.extract_team_df()

,Position,Club,GP,Won,Lost,Win%,PTS+,PTS-,+/-,H,A,OT,L10
0,1,Olympiacos Piraeus *,33,23,10,69.7%,2771,2500,+271,13-3,10-7,0-0,7-3
1,2,Real Madrid *,33,23,10,69.7%,2791,2580,+211,13-4,10-6,1-1,7-3
2,3,FC Barcelona *,33,22,11,66.7%,2638,2509,+129,12-4,10-7,1-1,7-3
3,4,AS Monaco *,33,21,12,63.6%,2730,2662,+68,13-4,8-8,2-1,7-3
4,5,Fenerbahce Beko Istanbul,33,19,14,57.6%,2744,2666,+78,12-5,7-9,2-0,5-5
5,6,Partizan Mozzart Bet Belgrade *,33,19,14,57.6%,2794,2700,+94,11-5,8-9,0-2,7-3
6,7,Maccabi Playtika Tel Aviv *,33,19,14,57.6%,2741,2657,+84,14-2,5-12,0-0,7-3
7,8,Cazoo Baskonia Vitoria-Gasteiz,33,18,15,54.5%,2841,2750,+91,14-3,4-12,2-0,5-5
8,9,Zalgiris Kaunas,33,18,15,54.5%,2508,2546,-38,13-4,5-11,0-0,6-4
9,10,Crvena Zvezda Meridianbet Belgrade,33,16,17,48.5%,2512,2534,-22,10-6,6-11,1-1,6-4


In [44]:
extractor.player_info_df()

all players expanded
https://www.euroleaguebasketball.net/euroleague/players/alberto-abalde/003733/
https://www.euroleaguebasketball.net/euroleague/players/awudu-abass/003527/
https://www.euroleaguebasketball.net/euroleague/players/veniamin-abosi/012298/
https://www.euroleaguebasketball.net/euroleague/players/alex-abrines/003478/
https://www.euroleaguebasketball.net/euroleague/players/jalen-adams/012056/


,name,team,Nationality,Height,Born,pic_link
0,ALBERTO ABALDE,Real Madrid,Spain,202,15 Dec 1995,https://media-cdn.incrowdsports.com/82c96970-c...
1,AWUDU ABASS,Virtus Segafredo Bologna,Italy,198,27 Jan 1993,https://media-cdn.incrowdsports.com/41895699-9...
2,VENIAMIN ABOSI,Olympiacos Piraeus,Greece,194,15 Jan 2006,None
3,ALEX ABRINES,FC Barcelona,Spain,198,1 Aug 1993,https://media-cdn.incrowdsports.com/e37b7de4-d...
4,JALEN ADAMS,Maccabi Playtika Tel Aviv,United States of America,191,11 Dec 1995,https://media-cdn.incrowdsports.com/14bb2746-d...


In [46]:
from pyspark.sql import SparkSession

In [7]:
def get_driver():
    service = Service(r'driver/chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [8]:
def extract_df(stats_link):
    driver = get_driver()
    driver.get(stats_link)
    time.sleep(5)
#     print(driver.page_source)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
    rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

    header_list = []
    for cell in rows[0]:
    #     print(cell.text, end= ' ')
        header_list.append(cell.text)

    data = []
    for row in rows:
        row_data = []
        for cell in row.find_all('div', {'role':'cell'}):
            if cell.find('a'):
                span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
    #             print(span.text, end=' ')
                row_data.append(span.text)
            else:
    #             print(cell.text, end=' ')
                row_data.append(cell.text)
    #     print('\n')
        data.append(row_data)
    return pd.DataFrame(data[1:], columns=header_list)

In [5]:
extract_df(player_uri)

,#,Player,Team,GP,Min,eFG%,TS%,OREB%,DREB%,REB%,AST/TO,AST-R,TO-R,2PTA-R,3PTA-R,FT-RT
0,1,ARTURS KURUCS,BKN,21,6:53,42.6%,43%,1.6%,2.4%,2%,1.4,30.6%,22.1%,15.3%,30.6%,7.4%
1,2,MATTHEW STRAZEL,ASM,20,7:20,45.2%,47.8%,0.8%,1.8%,1.2%,1.2,22.2%,18.8%,23.9%,29%,25.8%
2,3,SANDER RAIESTE,BKN,18,6:59,50%,48.2%,7.5%,11.2%,9.3%,1,14.3%,14.3%,40.1%,28.7%,8.3%
3,4,KAROLIS LUKOSIUNAS,ZAL,19,7:03,58.8%,58.8%,2.6%,4.7%,3.6%,1.5,13.6%,9.1%,6.8%,70.5%,0%
4,5,RAFI MENCO,MTA,20,5:37,44.4%,47.7%,9%,10.8%,9.9%,1.5,15.7%,10.4%,26.1%,44.4%,11.1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,204,NIKOLA MIROTIC,BAR,22,25:18,60.5%,65.1%,9.1%,14.1%,11.7%,0.8,8.2%,9.7%,37.3%,31.8%,43%
204,205,VASILIJE MICIC,EFS,29,31:29,53.7%,59.2%,1.3%,11%,6.3%,2,24.9%,12.8%,23.5%,31.2%,32%
205,206,WILL CLYBURN,EFS,32,33:15,52.2%,57.1%,3.9%,17.7%,11%,1.3,11.8%,8.8%,39.1%,31%,30%
206,207,MATHIAS LESSORT,PAR,32,30:35,68.9%,72.6%,14.3%,16.5%,15.4%,1,12.1%,12.4%,57.3%,0.3%,70.8%


### GET TEAM DATA

In [84]:
def extract_team_df(stats_link):
    driver = get_driver()
    driver.get(stats_link)
    time.sleep(5)
#     print(driver.page_source)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
    rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

    header_list = []
    for cell in rows[0].find_all('div', attrs={'class':'complex-stat-table_cell__1lxC7'}):
        span = cell.find('span', attrs={'class':'complex-stat-table_tabletHeadName__1Jn7u'})
        if span:
            header_list.append(span.text.replace('*','').strip())
        else:
            header_list.append(cell.find('span').text.replace('*','').strip())

    data = []
    for row in rows:
        row_data = []
        for cell in row.find_all('div', {'role':'cell'}):
            if cell.find('a'):
                span = cell.find('span', attrs={'class':'complex-stat-table_mainClubName__3IMZJ'})
    #             print(span.text, end=' ')
                row_data.append(span.text)
            else:
    #             print(cell.text, end=' ')
                row_data.append(cell.text)
    #     print('\n')
        data.append(row_data)
    return pd.DataFrame(data[1:], columns=header_list)

In [85]:
extract_team_df(standings)

,Position,Club,GP,Won,Lost,Win%,PTS+,PTS-,+/-,H,A,OT,L10
0,1,Olympiacos Piraeus *,32,23,9,71.9%,2692,2413,+279,13-3,10-6,0-0,8-2
1,2,FC Barcelona *,32,22,10,68.8%,2562,2425,+137,12-4,10-6,1-1,8-2
2,3,Real Madrid *,32,22,10,68.8%,2712,2513,+199,12-4,10-6,1-1,7-3
3,4,AS Monaco *,32,21,11,65.6%,2646,2574,+72,13-3,8-8,2-1,7-3
4,5,Maccabi Playtika Tel Aviv,32,19,13,59.4%,2674,2589,+85,14-2,5-11,0-0,8-2
5,6,Fenerbahce Beko Istanbul,32,18,14,56.2%,2641,2580,+61,11-5,7-9,2-0,5-5
6,7,Partizan Mozzart Bet Belgrade,32,18,14,56.2%,2706,2616,+90,11-5,7-9,0-2,7-3
7,8,Cazoo Baskonia Vitoria-Gasteiz,32,17,15,53.1%,2721,2650,+71,13-3,4-12,2-0,5-5
8,9,Zalgiris Kaunas,32,17,15,53.1%,2440,2479,-39,12-4,5-11,0-0,5-5
9,10,Anadolu Efes Istanbul,32,16,16,50%,2627,2561,+66,10-6,6-10,0-2,5-5


### GET PLAYER INFO

In [21]:
belinelli = 'https://www.euroleaguebasketball.net/euroleague/players/marco-belinelli/bcn/'

In [29]:
def player_info(player_link):
    driver = get_driver()
    driver.get(player_link)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source,'lxml')
    spans = soup.find('h1', attrs={'class':'hero-info_fullName__IDuVE'}).find_all('span')
    name = ''
    for span in spans:
        name += f' {span.text}'
    name = name.strip()
    info['name'] = name
    info['team'] = soup.find('h2', attrs={'class':'hero-info_teamName__3g98M'}).text
    for li in soup.find('ul', attrs={'class','hero-info_dataList__2BmgP'}):
        info[li.find('span').text] = li.find('b').text
    img = soup.find('div', attrs={'class':'player-hero_imageWrap__161BQ cover-contain_ofCover__3K_IS'}).find('img')
    info['pic_link'] = img['data-srcset'].split('?')[0]
    return info

In [30]:
player_info('https://www.euroleaguebasketball.net/euroleague/players/mathias-lessort/003842/')

{'name': 'MATHIAS LESSORT',
 'team': 'Partizan Mozzart Bet Belgrade',
 'Nationality': 'France',
 'Height': '206',
 'Born': '29 Sep 1995',
 'pic_link': 'https://media-cdn.incrowdsports.com/cb3024fc-3af4-4665-bd51-4ecb2965ae17.png'}

In [51]:
driver = get_driver()

In [52]:
driver.get('https://www.euroleaguebasketball.net/euroleague/players/')

In [53]:
load_more_xpath = '//*[@id="main"]/section/div[3]/div/div[4]/button'
load_more_class = 'all-players_loadMoreBtn__35J6M'
load_more_button = driver.find_element(by="class name", value=load_more_class)
while load_more_button: 
    load_more_button.click()
    print('clicked')
    time.sleep(2)
    try:
        load_more_button = driver.find_element(by="class name", value=load_more_class)
    except NoSuchElementException:
        print('all players expanded')
        load_more_button = None

clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
all players expanded


In [62]:
soup = BeautifulSoup(driver.page_source,'lxml')
player_list = soup.find('ul', attrs={'class':'all-players-list_filterList__1jH_E'})

In [73]:
home = 'https://www.euroleaguebasketball.net'
urls = []
for li in player_list.find_all('li'):
    path = urljoin(home, li.find('a')['href'])
    urls.append(path)
urls

['https://www.euroleaguebasketball.net/euroleague/players/alberto-abalde/003733/',
 'https://www.euroleaguebasketball.net/euroleague/players/awudu-abass/003527/',
 'https://www.euroleaguebasketball.net/euroleague/players/veniamin-abosi/012298/',
 'https://www.euroleaguebasketball.net/euroleague/players/alex-abrines/003478/',
 'https://www.euroleaguebasketball.net/euroleague/players/jalen-adams/012056/',
 'https://www.euroleaguebasketball.net/euroleague/players/tomer-agmon/010941/',
 'https://www.euroleaguebasketball.net/euroleague/players/dimitris-agravanis/004429/',
 'https://www.euroleaguebasketball.net/euroleague/players/ismet-akpinar/004056/',
 'https://www.euroleaguebasketball.net/euroleague/players/kyle-alexander/011973/',
 'https://www.euroleaguebasketball.net/euroleague/players/carlos-alocen/010623/',
 'https://www.euroleaguebasketball.net/euroleague/players/davide-alviti/005059/',
 'https://www.euroleaguebasketball.net/euroleague/players/mohammad-amini/012378/',
 'https://www.

In [4]:
pd.DataFrame([{'a':2, 'b':3}, {'a':1,'b':5}])

,a,b
0,2,3
1,1,5
